In [49]:
from balpy_v2.lib.gql import gql

GRAPH_URL = "https://api.thegraph.com/subgraphs/name/bleu-studio/balancer-mainnet-v2"
SWAPS_QUERY = """query {
  swaps(first: 1000, where:{poolId:"0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112", timestamp_gt:1657497600, timestamp_lt: 1658718000}, orderBy:timestamp) { 
    id
    valueUSD
    poolId {
      id
      swapFee
    }
  }  
}"""


In [50]:
import datetime
import asyncio
from balpy_v2.subgraphs.blocks import get_block_number_by_timestamp
from functools import reduce
REPORT_PERIOD_START_DATE = datetime.datetime(2022, 7, 11, 0, 0, 0).timestamp()
jul11_ts = REPORT_PERIOD_START_DATE
jul25_ts = jul11_ts + datetime.timedelta(weeks=2).total_seconds()
jul11_block, jul25_block = await asyncio.gather(
    *[
        get_block_number_by_timestamp(timestamp=int(ts))
        for ts in [jul11_ts, jul25_ts]

    ])
from balpy_v2.pools.fees_report import REPORT_CYCLES_START_DATES


swaps = await gql(
    GRAPH_URL,
    SWAPS_QUERY,
    variables=dict(block=jul25_block),
)

approx_swap_fees = reduce(lambda acc, curr: acc+float(curr["valueUSD"]), swaps["swaps"], 0) * 0.0004
print(f"Approximate swap fees: {approx_swap_fees}")

Approximate swap fees: 2333.0515199093857


In [51]:
JOINS_QUERY = """
query {
  joinExits(first: 1000, where:{pool:"0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112", timestamp_gt:1657497600, timestamp_lt: 1658718000}, orderBy:timestamp) {
    id
    timestamp
    valueUSD
    pool {
      id
    }
  }
}
"""


join_exits = await gql(
    GRAPH_URL,
    JOINS_QUERY,
    variables=dict(block=jul25_block),
)
join_exits["joinExits"].__len__()

25

In [52]:
approx_joins_exits = reduce(lambda acc, curr: acc+float(curr["valueUSD"]), join_exits["joinExits"], 0) * 0.0004
print(f"Approximate joins/exits fees: {approx_joins_exits}")

Approximate joins/exits fees: 1415.2606925737334


In [54]:
approx_joins_exits + approx_swap_fees

3748.312212483119

In [58]:
 await gql(
    GRAPH_URL,
    """query($block: Int!) {
    pool(id: "0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112", block: {number: $block}) {
    totalSwapFee
    }
    }""",
    variables=dict(block=jul11_block),
)

{'pool': {'totalSwapFee': '35390.34348398538671327838061504521'}}

In [59]:
37711.57913132734176101946688161184 - 35390.34348398538671327838061504521

2321.235647341957

In [ ]:
# Ok, let's pick reth/weth and see how much it paid and then compare to what Solar has in his spreadsheet
reth_weth = paid_fees[paid_fees["pool_symbol"] == "B-stETH-STABLE"]
# and now let's get the price of the token at the "current" time
token_addr, token_symbol, amount_in_token = reth_weth["address"][0], reth_weth["symbol"][0], reth_weth["paidProtocolFees_diff"][0]
token_addr, token_symbol, amount_in_token, reth_weth["token_latestUSDPrice_start"][0], amount_in_token * reth_weth["token_latestUSDPrice_start"][0]